# Prestage data for computation

In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import os
from subprocess import Popen, PIPE
import tarfile

import config

## Print storage locations

In [21]:
config.project_tmpdir

'/glade/scratch/mclong/tmp/calc-cache'

In [22]:
config.model_data_dir_root

'/glade/scratch/mclong/tmp/so-co2-airborne-obs/model-data'

In [23]:
config.model_data_dir

'/glade/scratch/mclong/tmp/so-co2-airborne-obs/model-data/Long-etal-2021-SO-CO2-Science'

In [24]:
config.dash_asset_fname

'Long-etal-2021-SO-CO2-Science.tar.gz'

## Get data from DASH repo

In [25]:
if not os.path.isdir(config.model_data_dir):
    # run wget to stage data
    # TODO: support curl too
    cwd = os.getcwd()
    script = f'{cwd}/wget-dash-archive.sh'

    os.chdir(config.model_data_dir_root)

    p = Popen(['bash', script], stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    if p.returncode:    
        print(stderr.decode('UTF-8'))
        print(stdout.decode('UTF-8'))
        raise OSError('data transfer failed')    

    # untar archive
    assert os.path.isfile(config.dash_asset_fname), f'missing {config.dash_asset_fname}'
    tar = tarfile.open(config.dash_asset_fname, "r:gz")
    tar.extractall()
    tar.close()

    os.chdir(cwd)

os.listdir(config.model_data_dir)

['TM5-Flux-mrf',
 'TM5-Flux-m0f',
 'CT2019B',
 's99oc_SOCCOM_v2020',
 's99oc_v2020',
 's99oc_ADJocI40S_v2020',
 'CAMSv20r1',
 'CT2017',
 'MIROC',
 'README.md',
 'CTE2018',
 'TM5-Flux-mmf',
 'TM5-Flux-mwf',
 'CTE2020']

## Check on `intake` datasets

In [26]:
import models

In [27]:
ds = models.dataset_som_ffn.open_dataset()